In [1]:
import numpy as np
import conectonator
import pygame as pg

pygame 2.6.1 (SDL 2.28.4, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
class rendererer:
    def __init__(
            self,
            res=(1280,720),
            frame_rate=60
        ):
        pg.init()
        self.screen = pg.display.set_mode(res)
        self.frame_rate = frame_rate
        self.clock = pg.time.Clock()
        self.running = False
    
    def run_display(self):
        self.running = True
        while self.running:
            for event in pg.event.get():
                if event.type == pg.QUIT:
                    self.running = False
            self.display_func()
        pg.quit()

    def display_func(self):
        self.screen.fill("purple")
        pg.display.flip()
        self.clock.tick(self.frame_rate)
        
    

In [17]:
def main():
    # game = connectanator()
    # print(game.get_board())
    # while True:
    #     game.game_turn()
    #     print(game.get_board())
    rend = rendererer()
    rend.run_display()

if __name__ == "__main__":
    main()